In [3]:
import pandas as pd
import numpy as np

# Indlæs filen
df_landbrug = pd.read_csv('landbrugsdata.csv')

#filter for løbende priser og foregående års priser
df_landbrug_loebende=df_landbrug.loc[df_landbrug['PRISENHED']=='Løbende priser']
df_landbrug_loebende=df_landbrug_loebende.loc[df_landbrug_loebende['TID']!=1966]
df_landbrug_for=df_landbrug.loc[df_landbrug['PRISENHED']!='Løbende priser']

#fjern prishenhed
df_landbrug_loebende.drop(columns=['PRISENHED'],inplace=True)
df_landbrug_for.drop(columns=['PRISENHED'],inplace=True)

#set index
df_landbrug_loebende.set_index(['TILGANG1','TILGANG2', 'ANVENDELSE', 'TID'],inplace=True)
df_landbrug_for.set_index(['TILGANG1','TILGANG2', 'ANVENDELSE', 'TID'],inplace=True)

#tjek
df_landbrug_loebende

C:\Users\b431385\AppData\Local\Temp\ipykernel_15664\2109001815.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_landbrug_for.drop(columns=['PRISENHED'],inplace=True)


INDHOLD
TILGANG1          TILGANG2                     ANVENDELSE                                         TID           
Import inkl. told 100010 Slagterier-(Tilgang)  Eksport - (Anvendelse)                             1980    43.260
                                               Lagre - (Anvendelse)                               1980     6.632
Dansk produktion  100010 Slagterier-(Tilgang)  Anvendelse, i alt-(Anvendelse)                     1967  6862.242
                                               Faste bruttoinvesteringer ialt - (Anvendelse)      1967    11.226
                                               Offentligt forbrug, i alt-(Anvendelse)             1967     0.008
...                                                                                                          ...
Told              REST_TILGANG Øvrige brancher 100040x100050 Anden fødevareindustri (100040, 1... 2018    13.293
                                                                                                  2019    12.384
                                                                                                  2020    13.085
                                                                                                  2021    15.983
                                                                                                  2022    19.360

[7150 rows x 1 columns]

In [4]:
# Beregn Pt/Pt-1
Pt_Pt=df_landbrug_loebende/df_landbrug_for

# Vi bruger .unstack() til at flytte 'TID' fra index til kolonner
df_wide = Pt_Pt['INDHOLD'].unstack(level='TID')

# Lav en tom tabel i samme form til vores resultat (Pt)
Pt = df_wide.copy()
Pt.loc[:, :] = np.nan  # Tøm tabellen for tal
Pt[2020] = 1.0         # Sæt 2020 til 1.0
# Find alle år i dit datasæt og sortér dem
years = sorted(df_wide.columns)

# FREMAD: Fra 2021 og op (Pt = Pt-1 * Pt_Pt_nu)
for y in years:
    if y > 2020:
        Pt[y] = Pt[y-1] * df_wide[y]

# BAGUD: Fra 2019 og ned (Pt = Pt+1 / Pt_Pt_næste_år)
for y in reversed(years):
    if y < 2020:
        Pt[y] = Pt[y+1] / df_wide[y+1]

df_Pt_final = Pt.stack(future_stack=True).reset_index()

# Navngiv kolonnerne som før
df_Pt_final.columns = ['TILGANG1', 'TILGANG2', 'ANVENDELSE', 'TID', 'Pt']
df_Pt_final


,TILGANG1,TILGANG2,ANVENDELSE,TID,Pt
0,Dansk produktion,010000 Landbrug og gartneri-(Tilgang),010000 Landbrug og gartneri- (Anvendelse),1967,0.202578
1,Dansk produktion,010000 Landbrug og gartneri-(Tilgang),010000 Landbrug og gartneri- (Anvendelse),1968,0.184537
2,Dansk produktion,010000 Landbrug og gartneri-(Tilgang),010000 Landbrug og gartneri- (Anvendelse),1969,0.195897
3,Dansk produktion,010000 Landbrug og gartneri-(Tilgang),010000 Landbrug og gartneri- (Anvendelse),1970,0.213069
4,Dansk produktion,010000 Landbrug og gartneri-(Tilgang),010000 Landbrug og gartneri- (Anvendelse),1971,0.214818
...,...,...,...,...,...
9235,Told,REST_TILGANG Øvrige brancher,REST_ANVENDELSE Øvrige brancher,2018,1.057831
9236,Told,REST_TILGANG Øvrige brancher,REST_ANVENDELSE Øvrige brancher,2019,1.011060
9237,Told,REST_TILGANG Øvrige brancher,REST_ANVENDELSE Øvrige brancher,2020,1.000000
9238,Told,REST_TILGANG Øvrige brancher,REST_ANVENDELSE Øvrige brancher,2021,1.181682


In [5]:
df_Pt_final.set_index(['TILGANG1','TILGANG2', 'ANVENDELSE', 'TID'],inplace=True)
df_Xt=df_landbrug_loebende['INDHOLD']/df_Pt_final['Pt']
df_Xt.columns = ['TILGANG1', 'TILGANG2', 'ANVENDELSE', 'TID', 'Xt']
df_Xt_final = df_Xt.to_frame(name='Xt')

# 2. Fjern MultiIndex så den bliver "flad" (ligesom din Pt tabel)
df_Xt_final = df_Xt_final.reset_index()

# 3. Nu vil den se præcis ud som dine andre tabeller
df_Xt_final
df_Xt_final.to_csv('landbrugsdata_faste_priser.csv', index=False)